\
            # 40. WebSocket 实战（WebSocket in Practice）

            讲清 WebSocket 的用途与基本协议，并用可选依赖 `websockets` 在本机写回显/广播示例。
依赖安装：`pip install websockets`（如果没装，Notebook 会自动跳过并给出提示）。

            > 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行（第三方依赖会做可选降级）。


## 前置知识

- asyncio 实战
- JSON（json 模块）
- 网络基础（TCP/HTTP）


## 知识点地图

- 1. WebSocket 是什么：一条连接上双向收发消息
- 2. 最小回显：本机 server/client
- 3. 消息协议设计：type/version/request_id/error
- 4. 心跳、重连与资源回收


## 自检清单（学完打勾）

- [ ] 知道 WebSocket 与 HTTP 的区别（长连接/双向）
- [ ] 能用 websockets 写最小 server/client
- [ ] 会设计简单消息协议（type 字段）并做输入校验
- [ ] 知道心跳/重连/背压的必要性


In [ ]:
\
from pathlib import Path

ART = Path('_nb_artifacts')
ART.mkdir(exist_ok=True)
print('artifacts dir:', ART.resolve())


## 知识点 1：WebSocket 是什么：一条连接上双向收发消息

- WebSocket 通过 HTTP Upgrade 握手建立连接，之后以帧（frame）双向通信。
- 适合：聊天室、协同编辑、实时通知、在线游戏等。
- 与轮询相比：减少重复握手与延迟；与 SSE 相比：支持双向。


## 知识点 2：最小回显：本机 server/client

说明：
- 依赖 `websockets`：`pip install websockets`
- Notebook 支持顶层 `await`；如在脚本里请改成 `asyncio.run(main())`


In [ ]:
import asyncio
import json

try:
    import websockets
except Exception as e:
    print('websockets not available:', type(e).__name__, e)
    print('install: pip install websockets')
else:
    async def handler(ws):
        async for msg in ws:
            data = {'type': 'echo', 'message': msg}
            await ws.send(json.dumps(data))

    async def main():
        async with websockets.serve(handler, '127.0.0.1', 8765):
            await asyncio.sleep(0.05)
            async with websockets.connect('ws://127.0.0.1:8765') as ws:
                await ws.send('hello')
                resp = await ws.recv()
                print('resp:', resp)

    await main()


## 知识点 3：消息协议设计：type/version/request_id/error

不要用“裸字符串协议”。推荐 JSON：
- `type`: 消息类型（chat/join/leave/ping/...）
- `payload`: 业务数据
- `request_id`: 可选，用于请求-响应关联
- `version`: 协议版本，便于演进

服务端必须做输入校验：字段缺失/类型不对要返回 error，而不是让服务端崩。


## 知识点 4：心跳、重连与资源回收

- 心跳：ping/pong 或应用层心跳，检测半断开连接。
- 重连：指数退避 + 抖动；避免所有客户端同时重连。
- 断线要清理连接集合，避免内存泄漏。
- 广播要考虑慢客户端：需要背压/丢弃策略/每连接发送队列。


## 常见坑

- 把 WebSocket 当 HTTP：每条消息都新建连接
- 不校验输入：任意 payload 导致服务端异常或安全问题
- 不做心跳：遇到 NAT/代理断线无法及时发现
- 广播时不做背压：一个慢客户端拖垮所有人


## 综合小案例：广播聊天室（2 个客户端，同一进程演示）

- 服务端维护在线连接集合
- 任意客户端发消息，服务端广播给所有连接
- 演示 2 个客户端在同一事件循环里连接并收发


In [ ]:
import asyncio
import json

try:
    import websockets
except Exception as e:
    print('websockets not available:', type(e).__name__, e)
    print('install: pip install websockets')
else:
    clients = set()

    async def handler(ws):
        clients.add(ws)
        try:
            async for msg in ws:
                payload = json.dumps({'type': 'chat', 'message': msg})
                await asyncio.gather(*[c.send(payload) for c in list(clients)])
        finally:
            clients.discard(ws)

    async def client(name):
        async with websockets.connect('ws://127.0.0.1:8766') as ws:
            await ws.send(f'{name}: hi')
            for _ in range(2):
                msg = await ws.recv()
                print(name, 'recv', msg)

    async def main():
        async with websockets.serve(handler, '127.0.0.1', 8766):
            await asyncio.sleep(0.05)
            await asyncio.gather(client('c1'), client('c2'))

    await main()


## 自测题（不写代码也能回答）

- 为什么 WebSocket 需要心跳？哪些场景会出现“半断开”？
- 为什么要设计带 type 的消息协议？
- 广播时如何避免单个慢客户端拖垮所有人？（背压/限流）


## 练习题（建议写代码）

- 给聊天室加上用户名注册与在线列表。
- 实现“只给指定房间广播”：room_id -> clients 映射。
- 为服务端加入简单鉴权：连接 URL 带 token 并校验（示意即可）。
